In [58]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy

import aot

In [59]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]
video_db_path = base_dir / "data/videos/database_originals.tsv"
video_db = pd.read_csv(video_db_path, sep="\t")
eyetracking_settings_path = settings_root_path / "eyetracking"
subject_number = core_settings["various"]["subject_number_eyetracking"]
session_number = core_settings["various"]["session_number_eyetracking"]
run_number = core_settings["various"]["run_number_eyetracking"]
blank_number_per_run = core_settings["various"][
    "blank_number_per_run"
]  # only for main run
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
video_number_per_run_main = int((unique_video_number_per_session * 4) / run_number)

videos_names = []
for i in range(1, len(video_db)):
    if (
        video_db["grade"][i] != "k" and video_db["grade"][i] != "l"
    ):  # video_db['grade'][i] == 'j' or video_db['grade'][i] == 'NA':  
        videos_names.append(video_db["video_name"][i])
print("videos number: ", len(videos_names))

origin_videos_number = len(videos_names)

video_number_per_run_grading = int(
    origin_videos_number / run_number
)  # only 1 subject and 1 session for grading

videos number:  1548


In [60]:
def read_settings_file(subject, ses, run):
    settings_file = (
        eyetracking_settings_path
        / f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(ses).zfill(2)}_run_{str(run).zfill(2)}.yml"
    )
    settings = yaml.load(open(settings_file), Loader=yaml.FullLoader)
    return settings

In [61]:
def read_settings_files_session(
    subject, ses
):  # return a list of settings files for a given session
    settings_files = [
        read_settings_file(subject, ses, run) for run in range(1, run_number + 1)
    ]
    sessions_movies = [
        settings["stimuli"]["movie_files"] for settings in settings_files
    ]
    return sessions_movies

In [62]:
def check_uniqe_video_number_bothdirection(sub, ses):
    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    print(videoset)
    print("unique video number: ", len(videoset))
    return videoset


def check_uniqe_video_number_uniqe_original(sub, ses):
    def deR(name):
        return name[2:]
    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    new_video_list = []
    for video in videoset:
        if video[0] == "R":
            new_video_list.append(deR(video))
        else:
            new_video_list.append(video) 
    
    videoset = set(new_video_list)
    #print(videoset)
    print("unique video number sub {sub} ses {ses}: ".format([sub,ses]), len(videoset))
   
    return videoset

In [63]:
# check_uniqe_video_number_bothdirection(1, 3)
check_uniqe_video_number_uniqe_original(1, 3)
for sub in range(1, subject_number + 1):
    for ses in range(1, session_number + 1):
        check_uniqe_video_number_uniqe_original(sub, ses)

KeyError: 'sub'